In [2]:
from keras.datasets import reuters
(train_data,train_labels),(test_data,test_labels) = reuters.load_data(num_words=10000)

/Users/garethjones/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:955: UserWarning: Illegal line #3
	""""
"
	in file "/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)
/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:955: UserWarning: Illegal line #7
	""""
"
	in file "/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)

Bad key "Created on Thu May 24 09" on line 4 in
/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotl

In [3]:
import numpy as np
print("Training dataset is shape: ",train_data.shape)
print("Training labels is shape: ",train_labels.shape)
print("Testing dataset is shape: ",test_data.shape)
print("Testing labels is shape: ",test_labels.shape)

# find lengths of all the samples
x = [len(i) for i in train_data]
print("The max length of a single sample is ",max(x))
print("The min length of a single sample is ",min(x))
print("The avg length of all samples is ",round(np.mean(x)))

train_labels

Training dataset is shape:  (8982,)
Training labels is shape:  (8982,)
Testing dataset is shape:  (2246,)
Testing labels is shape:  (2246,)
The max length of a single sample is  2376
The min length of a single sample is  13
The avg length of all samples is  146.0


array([ 3,  4,  3, ..., 25,  3, 25])

In [46]:
# I want to have a look to see what the data actually is
# I need to turn by keys into values using my reverse word index
train_data[0]

#decoded_train_data1 = []
#for i in train_data[0]:
#    x = reverse_word_index.get(i-3,'?')
#    decoded_train_data1.append(x)
#print(decoded_train_data1)

# get good at this list enumeration thing!
decoded_train_data = [reverse_word_index.get(i-3,'?') for i in train_data[3]]
reuters_message = " ".join(i for i in decoded_train_data)
print(reuters_message)

# now I want to see how many categories there are for each sample
print(max(train_labels))




? the farmers home administration the u s agriculture department's farm lending arm could lose about seven billion dlrs in outstanding principal on its severely ? borrowers or about one fourth of its farm loan portfolio the general accounting office gao said in remarks prepared for delivery to the senate agriculture committee brian crowley senior associate director of gao also said that a preliminary analysis of proposed changes in ? financial eligibility standards indicated as many as one half of ? borrowers who received new loans from the agency in 1986 would be ? under the proposed system the agency has proposed evaluating ? credit using a variety of financial ratios instead of relying solely on ? ability senate agriculture committee chairman patrick leahy d vt ? the proposed eligibility changes telling ? administrator ? clark at a hearing that they would mark a dramatic shift in the agency's purpose away from being farmers' lender of last resort toward becoming a big city bank but 

In [57]:
x = np.array(train_labels)
unique, counts = np.unique(x, return_counts=True)
print(unique, counts)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]), array([  55,  432,   74, 3159, 1949,   17,   48,   16,  139,  101,  124,
        390,   49,  172,   26,   20,  444,   39,   66,  549,  269,  100,
         15,   41,   62,   92,   24,   15,   48,   19,   45,   39,   32,
         11,   50,   10,   49,   19,   19,   24,   36,   30,   13,   21,
         12,   18]))


In [68]:
word_index = reuters.get_word_index()
print(len(word_index))

# create the reverse of the word_index so I can match to the original dataset
reverse_word_index = dict([(keys,values) for (values,keys) in word_index.items()])

# now I need to encode the samples in my dataset by vectorising each one. I can create a function to do this
# this creates a 2D numpy array of 1s and 0s showing each word in the sample corresponding to each word index
def vectorise_sequences(sequence,dimension=10000):
    result = np.zeros((len(sequence),dimension))
    for i, value in enumerate(sequence):
        result[i,value] = 1
    return result

# pass train and test samples to the function to get my x_train / test data
x_train = vectorise_sequences(train_data)
x_test = vectorise_sequences(test_data)

# be sure about the shape of the data that we're dealing with
# the output is 10000 long because we said only use the first 10,000 words when we imported the dataset
print(x_train.shape)
print(x_test.shape)

# the model was failing because I was showing the it an output of size one, but it was expecting it to be 46 (one-hot encoded)
print('Original output shape: ',y_train.shape)


30979
(8982, 10000)
(2246, 10000)
Original output shape:  (8982,)


In [65]:
from keras import models, layers
model = models.Sequential()
model.add(layers.Dense(16,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(46,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 16)                160016    
_________________________________________________________________
dense_46 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_47 (Dense)             (None, 45)                765       
Total params: 161,053
Trainable params: 161,053
Non-trainable params: 0
_________________________________________________________________


In [66]:
from keras import optimizers,losses,metrics
#model.compile(optimizer=optimizers.RMSprop(),
#             loss=losses.binary_crossentropy(),
#             metrics=[metrics.accuracy()])
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])

In [67]:
history = model.fit(x_train,y_train,epochs=5,batch_size=512)

ValueError: Error when checking target: expected dense_47 to have shape (45,) but got array with shape (1,)